In [1]:
import zmq
from datetime import datetime
import plotly.graph_objects as go

In [2]:
symbol = 'SYMBOL'

In [3]:
fig = go.FigureWidget()
fig.add_scatter()
fig

FigureWidget({
    'data': [{'type': 'scatter', 'uid': '980da929-60cb-4635-b2b3-c3de2956f0e7'}], 'layout': {'t…

In [4]:
context = zmq.Context()

In [5]:
socket = context.socket(zmq.SUB)

In [6]:
socket.connect('tcp://0.0.0.0:5555')

<SocketContext(connect='tcp://0.0.0.0:5555')>

In [7]:
socket.setsockopt_string(zmq.SUBSCRIBE, 'SYMBOL')

In [8]:
times = list()
prices = list()

In [9]:
for _ in range(50):
    msg = socket.recv_string()
    t = datetime.now()
    times.append(t)
    _, price = msg.split()
    prices.append(float(price))
    fig.data[0].x = times
    fig.data[0].y = prices

In [10]:
fig = go.FigureWidget()
fig.add_scatter(name='SYMBOL')
fig.add_scatter(name='SMA1', line=dict(width=1, dash='dot'),
               mode='lines+markers')
fig.add_scatter(name='SMA2', line=dict(width=1, dash='dot'),
               mode='lines+markers')
fig

FigureWidget({
    'data': [{'name': 'SYMBOL', 'type': 'scatter', 'uid': '98563bb2-d223-4a69-a2e1-e475c3f5f46f…

In [11]:
import pandas as pd

In [12]:
df = pd.DataFrame()

In [13]:
for _ in range(75):
    msg = socket.recv_string()
    t = datetime.now()
    sym, price = msg.split()
    df = df.append(pd.DataFrame({sym: float(price)}, index=[t]))
    df['SMA1'] = df[sym].rolling(5).mean()
    df['SMA2'] = df[sym].rolling(10).mean()
    fig.data[0].x = df.index
    fig.data[1].x = df.index
    fig.data[2].x = df.index
    fig.data[0].y = df[sym]
    fig.data[1].y = df['SMA1']
    fig.data[2].y = df['SMA2']

In [14]:
from plotly.subplots import make_subplots

In [15]:
f = make_subplots(rows = 3, cols = 1, shared_xaxes = True)
f.append_trace(go.Scatter(name='SYMBOL'), row=1, col=1)
f.append_trace(go.Scatter(name='RETURN', line=dict(width=1, dash='dot'),
              mode='lines+markers', marker={'symbol': 'triangle-up'}),
               row=2, col=1)
f.append_trace(go.Scatter(name='MOMENTUM', line=dict(width=1, dash='dash'),
                         mode='lines+markers', marker={'symbol': 'x'}), 
               row=3, col=1)

In [16]:
fig = go.FigureWidget(f)

In [17]:
fig

FigureWidget({
    'data': [{'name': 'SYMBOL',
              'type': 'scatter',
              'uid': '20eebda9…

In [18]:
import numpy as np

In [19]:
df = pd.DataFrame()

In [20]:
for _ in range(75):
    msg = socket.recv_string()
    t = datetime.now()
    sym, price = msg.split()
    df = df.append(pd.DataFrame({sym: float(price)}, index=[t]))
    df['RET'] = np.log(df[sym] / df[sym].shift(1))
    df['MOM'] = df['RET'].rolling(10).mean()
    fig.data[0].x = df.index
    fig.data[1].x = df.index
    fig.data[2].x = df.index
    fig.data[0].y = df[sym]
    fig.data[1].y = df['RET']
    fig.data[2].y = df['MOM']